# Efficient NumPy

In [2]:
import numpy as np

## Best practices

### Avoid loops

Python loops are costly:

In [4]:
def square_loop(a):
    """Calculate square of an array in loop. We assume 1D array here."""

    result = np.zeros_like(a)
    for i in range(a.shape[0]):
        result[i] = a[i]*a[i]
    return result

In [5]:
large_arr = np.random.randint(100, size=(100000,))

In [6]:
%timeit -n 10 -r 3 square_loop(large_arr)

33.9 ms ± 481 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [7]:
%timeit -n 10 -r 3 np.square(large_arr)

118 µs ± 42.7 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


### Use broadcasting

Broadcasting mechanism provides an extremely efficient way of handling operations on arrays of different dimensionality. And it's always way more readable and concise. For example, to add `1D` array `b` to `2D` array `a` row-wise with a loop:

In [ ]:
def row_loop(a, b):
    """Add a vector to a matrix directly."""

    result = np.zeros_like(a)
    for i in range(a.shape[0]):
        result[i] = a[i] + b
    return result

In [ ]:
large_arr = np.random.randint(100, size=(1000,1000))
large_b = np.random.randint(100, size=(1000,))

In [ ]:
%timeit -n 10 -r 3 row_loop(large_arr, large_b)

Broadcasting is about `2X` faster:

In [ ]:
%timeit -n 10 -r 3 large_arr + large_b

In-place addition with broadcasting is even faster:

In [ ]:
%timeit -n 10 -r 3 np.add(large_arr, large_b, out=large_arr)

Btw, broadcasting allows for creating fancy structures in just a single line (you may leverage this in one of the problems in Homework #2):

In [ ]:
np.arange(10) + np.expand_dims(np.arange(10), axis=-1)

### Beware!

In-place operations are prone to bugs due to incorrect shape of the result container:

In [8]:
A = np.random.randint(10, size=(10,10))
B = np.random.randint(10, size=(10,))

In [9]:
A

array([[7, 7, 5, 3, 4, 1, 0, 7, 7, 7],
       [4, 4, 7, 7, 6, 4, 2, 4, 1, 7],
       [1, 0, 8, 0, 9, 0, 9, 7, 4, 4],
       [7, 5, 9, 4, 9, 8, 4, 0, 8, 0],
       [4, 6, 8, 6, 1, 8, 0, 5, 7, 0],
       [9, 5, 8, 7, 8, 7, 7, 7, 5, 7],
       [4, 4, 2, 6, 1, 4, 7, 6, 0, 5],
       [7, 8, 3, 3, 3, 7, 4, 8, 2, 0],
       [9, 3, 9, 6, 4, 0, 4, 3, 1, 1],
       [3, 3, 7, 6, 8, 8, 5, 1, 5, 1]])

In [10]:
B

array([9, 1, 4, 1, 3, 8, 9, 8, 2, 9])

In [11]:
A+B

array([[16,  8,  9,  4,  7,  9,  9, 15,  9, 16],
       [13,  5, 11,  8,  9, 12, 11, 12,  3, 16],
       [10,  1, 12,  1, 12,  8, 18, 15,  6, 13],
       [16,  6, 13,  5, 12, 16, 13,  8, 10,  9],
       [13,  7, 12,  7,  4, 16,  9, 13,  9,  9],
       [18,  6, 12,  8, 11, 15, 16, 15,  7, 16],
       [13,  5,  6,  7,  4, 12, 16, 14,  2, 14],
       [16,  9,  7,  4,  6, 15, 13, 16,  4,  9],
       [18,  4, 13,  7,  7,  8, 13, 11,  3, 10],
       [12,  4, 11,  7, 11, 16, 14,  9,  7, 10]])

In [12]:
np.add(A, B)

array([[16,  8,  9,  4,  7,  9,  9, 15,  9, 16],
       [13,  5, 11,  8,  9, 12, 11, 12,  3, 16],
       [10,  1, 12,  1, 12,  8, 18, 15,  6, 13],
       [16,  6, 13,  5, 12, 16, 13,  8, 10,  9],
       [13,  7, 12,  7,  4, 16,  9, 13,  9,  9],
       [18,  6, 12,  8, 11, 15, 16, 15,  7, 16],
       [13,  5,  6,  7,  4, 12, 16, 14,  2, 14],
       [16,  9,  7,  4,  6, 15, 13, 16,  4,  9],
       [18,  4, 13,  7,  7,  8, 13, 11,  3, 10],
       [12,  4, 11,  7, 11, 16, 14,  9,  7, 10]])

This one will work:

In [14]:
np.multiply(A, B, out=A)

array([[63,  7, 20,  3, 12,  8,  0, 56, 14, 63],
       [36,  4, 28,  7, 18, 32, 18, 32,  2, 63],
       [ 9,  0, 32,  0, 27,  0, 81, 56,  8, 36],
       [63,  5, 36,  4, 27, 64, 36,  0, 16,  0],
       [36,  6, 32,  6,  3, 64,  0, 40, 14,  0],
       [81,  5, 32,  7, 24, 56, 63, 56, 10, 63],
       [36,  4,  8,  6,  3, 32, 63, 48,  0, 45],
       [63,  8, 12,  3,  9, 56, 36, 64,  4,  0],
       [81,  3, 36,  6, 12,  0, 36, 24,  2,  9],
       [27,  3, 28,  6, 24, 64, 45,  8, 10,  9]])

This one will note (although broadcasting mechanics is ok for addition):

In [13]:
np.add(A, B, out=B)

ValueError: non-broadcastable output operand with shape (10,) doesn't match the broadcast shape (10,10)